In [70]:
#### Ethnicity
import pandas as pd
import sqlite3
from contextlib import closing
import psycopg2

nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        # Prepare the SQL query with format string for nct_ids
        query = f"""
            SELECT nct_id, result_group_id, ctgov_group_code, param_value, classification, category
            FROM baseline_measurements
            WHERE (title = 'Race (NIH/OMB)' OR title = 'Race/Ethnicity, Customized')
            AND nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])});
        """
        cursor.execute(query)
        patients = cursor.fetchall()

    columns = [
        'nct_id',
        'result_group_id',
        'ctgov_group_code',
        'number',
        'classification',
        'category',
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    #print(nunique)
    return(df)
    



extracted_race= extract_aact(nct_id_list)
#print(extracted_race)


max_result_group_id = extracted_race.groupby('nct_id')['result_group_id'].transform('max')

# Select rows where 'result_group_id' is equal to the max within its group
filtered_extracted_race = extracted_race[extracted_race['result_group_id'] == max_result_group_id]

print(filtered_extracted_race)

filtered_extracted_race.loc[filtered_extracted_race['category'].isnull(), 'category'] = filtered_extracted_race['classification']

category_mapping = {
    'Black': 'Black or African American',
    'BLACK OR AFRICAN AMERICAN' : 'Black or African American',
    'ASIAN' : 'Asian',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Native Hawaiian or Other Pacific Islander',
    'White or Caucasian': 'White',
    'AMERICAN INDIAN OR ALASKA NATIVE': 'American Indian or Alaska Native',
    'WHITE': 'White',
    'Unknown': 'Unknown or Not Reported',
    'Missing': 'Unknown or Not Reported',
    'Other': 'Unknown or Not Reported',
    'OTHER' : 'Unknown or Not Reported',
    'MISSING': 'Unknown or Not Reported',
    'No ethnicity': 'Unknown or Not Reported',
    'Other, not specified': 'Unknown or Not Reported',
    'Multiple': 'Unknown or Not Reported'
}

categories_to_remove = ["HISPANIC OR LATINO", "NOT HISPANIC OR LATINO","NOT KNOWN, NOT RECORDED, OR REFUSED"]

filtered_extracted_race['category'] = filtered_extracted_race['category'].replace(category_mapping)
filtered_extracted_race = filtered_extracted_race[~filtered_extracted_race['category'].isin(categories_to_remove)]
filtered_extracted_race = filtered_extracted_race.drop('classification', axis=1)
# Specify the file name for your CSV
race = "filtered_extracted_race.csv"

# Save the DataFrame to a CSV file
filtered_extracted_race.to_csv(race, index=False)




          nct_id  result_group_id ctgov_group_code number  \
2    NCT03167619         74624646            BG002      0   
5    NCT03167619         74624646            BG002     34   
8    NCT03167619         74624646            BG002      0   
11   NCT03167619         74624646            BG002      0   
14   NCT03167619         74624646            BG002      9   
..           ...              ...              ...    ...   
390  NCT02032823         74616130            BG002     48   
393  NCT02032823         74616130            BG002      1   
396  NCT02032823         74616130            BG002   1225   
399  NCT02032823         74616130            BG002      9   
402  NCT02032823         74616130            BG002     18   

                                classification  \
2                                         None   
5                                         None   
8                                         None   
11                                        None   
14               

/var/folders/64/x7_jk0ds5174rzhfbs9sv8b80000gn/T/ipykernel_32185/582654760.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_extracted_race['category'] = filtered_extracted_race['category'].replace(category_mapping)


In [51]:
nct_id_set = set(nct_id_list)
extracted_nct_id_set = set(extracted_race['nct_id'])

# Find nct_ids in nct_id_list but not in extracted_race
missing_nct_ids = nct_id_set - extracted_nct_id_set
print("nct_ids in the list but not in the extracted data:", missing_nct_ids)

nct_ids in the list but not in the extracted data: set()


nct_ids in the list but not in the extracted data: {'NCT02034916', 'NCT00679783'}
2 studies no ethnicities

In [52]:
### Age
nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact_age(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        query = f"""
        SELECT nct_id, result_group_id, classification, category, title, param_value
        FROM baseline_measurements
        WHERE (title = 'Age, Continuous' OR title = 'Age, Customized' OR title = 'Age, Categorical')
        AND nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])});;
        """
        cursor.execute(query)
        patients = cursor.fetchall()
        
        columns = [
        'nct_id',
        'result_group_id',
        'classification',
        'category',
        'title',
        'param_value'
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    print(nunique)
    return(df)
    



extracted_age= extract_aact_age(nct_id_list)
print(extracted_age)


max_result_group_id = extracted_age.groupby('nct_id')['result_group_id'].transform('max')

# Select rows where 'result_group_id' is equal to the max within its group
filtered_extracted_age = extracted_age[extracted_age['result_group_id'] == max_result_group_id]

print(filtered_extracted_age)


# Specify the file name for your CSV
age = "filtered_extracted_age.csv"

# Save the DataFrame to a CSV file
filtered_extracted_age.to_csv(age, index=False)




12
          nct_id  result_group_id classification     category  \
0    NCT03167619         74624644           None         None   
1    NCT03167619         74624645           None         None   
2    NCT03167619         74624646           None         None   
3    NCT01905592         74040544           None  18-64 years   
4    NCT01905592         74040545           None  18-64 years   
..           ...              ...            ...          ...   
147  NCT02032823         74616129    60-69 years         None   
148  NCT02032823         74616130    60-69 years         None   
149  NCT02032823         74616128     >=70 years         None   
150  NCT02032823         74616129     >=70 years         None   
151  NCT02032823         74616130     >=70 years         None   

               title param_value  
0    Age, Continuous        52.4  
1    Age, Continuous        49.5  
2    Age, Continuous          51  
3    Age, Customized          67  
4    Age, Customized         127  
..    

In [53]:
nct_id_set = set(nct_id_list)
extracted_nct_id_set = set(extracted_age['nct_id'])

# Find nct_ids in nct_id_list but not in extracted_race
missing_nct_ids = nct_id_set - extracted_nct_id_set
print("nct_ids in the list but not in the extracted data:", missing_nct_ids)

nct_ids in the list but not in the extracted data: set()


nct_ids in the list but not in the extracted data: {'NCT03343054', 'NCT01818063', 'NCT02734004', 'NCT02657889', 'NCT01905592', 'NCT04690855', 'NCT01506609'}

1st study: Age, customized in title, : <18,18-44, 45-64, >65 in classification
2nd study: Age, categorical in title, , <18, 18-65, >65, in category
3rd study: Age, categorical in title, <18, 18-65, >65, in category
4th study: Age, categorical in title:  as above
5th study: Age, customized: 18-64, 65-74, >75 in category
6th study: Age, categorical in title in category
7t study: Age, customized, <45, 45-64, >65 in category






In [54]:
### Age with custom and categorical
nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact_age_extra(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        query = f"""
        SELECT nct_id, result_group_id, classification, category, title, param_value
        FROM baseline_measurements
        WHERE (title = 'Age, Continuous' OR title = 'Age, Customized' OR title = 'Age, Categorical')
        AND nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])});;
        """
        cursor.execute(query)
        patients = cursor.fetchall()
        
        columns = [
        'nct_id',
        'result_group_id',
        'classification',
        'category',
        'title',
        'number',
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    print(nunique)
    return(df)
    


extracted_age_extra= extract_aact_age_extra(nct_id_list)
print(extracted_age_extra)


max_result_group_id = extracted_age_extra.groupby('nct_id')['result_group_id'].transform('max')

# Select rows where 'result_group_id' is equal to the max within its group
filtered_extracted_age_extra = extracted_age_extra[extracted_age_extra['result_group_id'] == max_result_group_id]

# Specify the file name for your CSV
age_extra = "filtered_extracted_age_extra.csv"

# Save the DataFrame to a CSV file
filtered_extracted_age_extra.to_csv(age_extra, index=False)


12
          nct_id  result_group_id classification     category  \
0    NCT03167619         74624644           None         None   
1    NCT03167619         74624645           None         None   
2    NCT03167619         74624646           None         None   
3    NCT01905592         74040544           None  18-64 years   
4    NCT01905592         74040545           None  18-64 years   
..           ...              ...            ...          ...   
147  NCT02032823         74616129    60-69 years         None   
148  NCT02032823         74616130    60-69 years         None   
149  NCT02032823         74616128     >=70 years         None   
150  NCT02032823         74616129     >=70 years         None   
151  NCT02032823         74616130     >=70 years         None   

               title number  
0    Age, Continuous   52.4  
1    Age, Continuous   49.5  
2    Age, Continuous     51  
3    Age, Customized     67  
4    Age, Customized    127  
..               ...    ...  
147  A

In [55]:
### trial design

nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact_design(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        query = f"""
        SELECT nct_id, allocation, intervention_model, masking, subject_masked, caregiver_masked, investigator_masked, outcomes_assessor_masked
        FROM designs
        WHERE nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])});
        """
        cursor.execute(query)
        patients = cursor.fetchall()
        
        columns = [
        'nct_id',
        'allocation',
        'intervention_model',
        'masking',
        'subject_masked',
        'caregiver_masked',
        'investigator_masked',
        'outcomes_assessor_masked'
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    print(nunique)
    return(df)
    



extracted_designs= extract_aact_design(nct_id_list)
print(extracted_designs)

# Specify the file name for your CSV
designs = "extracted_designs.csv"

# Save the DataFrame to a CSV file
extracted_designs.to_csv(designs, index=False)




12
         nct_id  allocation    intervention_model            masking  \
0   NCT01116648  Randomized   Parallel Assignment  None (Open Label)   
1   NCT01306032  Randomized  Crossover Assignment  None (Open Label)   
2   NCT01506609  Randomized   Parallel Assignment             Double   
3   NCT01818063  Randomized   Parallel Assignment  None (Open Label)   
4   NCT01905592  Randomized   Parallel Assignment  None (Open Label)   
5   NCT01945775  Randomized   Parallel Assignment  None (Open Label)   
6   NCT02000622  Randomized   Parallel Assignment  None (Open Label)   
7   NCT02032823  Randomized   Parallel Assignment             Triple   
8   NCT02163694  Randomized   Parallel Assignment             Double   
9   NCT02282020  Randomized                  None  None (Open Label)   
10  NCT03167619  Randomized   Parallel Assignment  None (Open Label)   
11  NCT03330847  Randomized   Parallel Assignment  None (Open Label)   

   subject_masked caregiver_masked investigator_masked  \
0 

In [56]:

### interventions

nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact_intervention(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        query = f"""
        SELECT d.nct_id, d.intervention_id, i.name, i.description, de.title
        FROM design_group_interventions d
        LEFT JOIN interventions i on d.intervention_id=i.id
        LEFT JOIN design_groups de on de.id=d.design_group_id
        WHERE d.nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])});
        """
        cursor.execute(query)
        patients = cursor.fetchall()
        
        columns = [
        'nct_id',
        'intervention_id',
        'name',
        'description',
        'title'
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    print(nunique)
    return(df)
    



extracted_intervention= extract_aact_intervention(nct_id_list)
print(extracted_intervention)



# Specify the file name for your CSV
interventions = "extracted_interventions.csv"

# Save the DataFrame to a CSV file
extracted_intervention.to_csv(interventions, index=False)



12
         nct_id  intervention_id                             name  \
0   NCT01818063         73676220                       Paclitaxel   
1   NCT01818063         73676220                       Paclitaxel   
2   NCT01818063         73676221                      Carboplatin   
3   NCT01818063         73676221                      Carboplatin   
4   NCT01818063         73676222                      Doxorubicin   
..          ...              ...                              ...   
57  NCT02163694         73643611                      Carboplatin   
58  NCT02163694         73643612                       Paclitaxel   
59  NCT02163694         73643612                       Paclitaxel   
60  NCT02000622         74000274                         Olaparib   
61  NCT02000622         74000275  Physician's choice chemotherapy   

                                          description  \
0                                            Given IV   
1                                            Given IV 

In [57]:
#### sex


nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact_sex(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        # Prepare the SQL query with format string for nct_ids
        query = f"""
            SELECT nct_id, result_group_id, ctgov_group_code, category,title, param_value
            FROM baseline_measurements
            WHERE (title ='Sex: Female, Male' )
            AND nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])});
        """
        cursor.execute(query)
        patients = cursor.fetchall()

    columns = [
        'nct_id',
        'result_group_id',
        'ctgov_group_code',
        'category',
        'title',
        'param_value'
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    #print(nunique)
    return(df)
    



extracted_sex= extract_aact_sex(nct_id_list)
#print(extracted_race)


max_result_group_id = extracted_sex.groupby('nct_id')['result_group_id'].transform('max')

# Select rows where 'result_group_id' is equal to the max within its group
filtered_extracted_sex = extracted_sex[extracted_sex['result_group_id'] == max_result_group_id]



# Specify the file name for your CSV
sex = "filtered_extracted_sex.csv"

# Save the DataFrame to a CSV file
filtered_extracted_sex.to_csv(sex, index=False)



In [62]:
#### outcomes


nct_id_list = ['NCT01306032', 'NCT02163694', 'NCT02282020', 'NCT02000622', 'NCT02032823', 
               'NCT03167619', 'NCT01818063', 'NCT01905592', 'NCT01116648', 'NCT01945775', 
               'NCT01506609', 'NCT03330847']

def extract_aact_outcomes(nct_ids):
    connection = psycopg2.connect(
        database="aact",
        user= "achang",
        password="AChang2608",
        host="aact-db.ctti-clinicaltrials.org",
    )

    with closing(connection.cursor()) as cursor:
        # Prepare the SQL query with format string for nct_ids
        query = f"""
            SELECT nct_id, outcome_type, measure
            FROM design_outcomes
            WHERE nct_id IN ({', '.join(["'%s'" % id for id in nct_ids])}) ;
        """
        cursor.execute(query)
        patients = cursor.fetchall()

    columns = [
        'nct_id',
        'outcome_type',
        'measure'
    ]

    df = pd.DataFrame(patients, columns=columns)

    nunique=df['nct_id'].nunique()
    unique_nct_ids = df['nct_id'].unique()
    #print(unique_nct_ids)
    #print(nunique)
    return(df)

extracted_outcomes= extract_aact_outcomes(nct_id_list)
    
print(extracted_outcomes)

          nct_id outcome_type  \
0    NCT01905592      primary   
1    NCT01905592    secondary   
2    NCT01905592    secondary   
3    NCT01905592    secondary   
4    NCT01905592    secondary   
..           ...          ...   
110  NCT02000622        other   
111  NCT02000622        other   
112  NCT01818063      primary   
113  NCT01818063    secondary   
114  NCT01818063    secondary   

                                               measure  
0    Progression Free Survival (PFS) - Central Revi...  
1                                     Overall Survival  
2    Number of Participants With Central BRCA Mutat...  
3    Number of Participants With Serious Adverse Ev...  
4    Progression Free Survival (PFS) - Investigator...  
..                                                 ...  
110  Time to First Subsequent Cancer Therapy or Dea...  
111  Time to Second Subsequent Cancer Therapy or De...  
112  Count of Participants That Achieve Pathologic ...  
113                          Over